In [1]:
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q accelerate
!pip install -q trl
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# !pip install --upgrade pyarrow

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [5]:
# General parameters
model_name = "Pennlaine/Mistral-7B-v02-Entity-Extraction"  # The model that you want to train from the Hugging Face hub
new_model = "mistralee-prescription-json-generator"  # The name for fine-tuned LoRA Adaptor

In [6]:
# LoRA parameters
lora_r = 64
lora_alpha = lora_r * 2
lora_dropout = 0.1
target_modules = ["q_proj", "v_proj", "k_proj"]

In [7]:
# QLoRA parameters
load_in_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant = False

In [21]:
# TrainingArguments parameters
num_train_epochs = 5
fp16 = False
bf16 = False
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
learning_rate = 0.00015
weight_decay = 0.01
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 10

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# Dataset parameters
use_special_template = True
response_template = " ### Answer:"
instruction_prompt_template = '"### Human:"'
use_llama_like_model = True

In [22]:
def load_data(file_path):
    percent_of_train_dataset = 0.90
    data = load_dataset("json", data_files=file_path, split="train")

    split_dataset = data.train_test_split(
        train_size=int(data.num_rows * percent_of_train_dataset), seed=19, shuffle=False
    )
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(eval_dataset)}")
    return train_dataset, eval_dataset

dataset_dict = {"seizure": {}, "prescription": {}}
dataset_dict["prescription"]["train_dataset"], dataset_dict["prescription"]["eval_dataset"] = load_data(
    file_path="/content/prescription.jsonl")

# dataset_dict["seizure"]["train_dataset"], dataset_dict["seizure"]["eval_dataset"] = load_data(
#     file_path="/content/seizure_frequency.jsonl")

In [23]:
# Load LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

In [24]:
# Load QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

In [25]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config, device_map=device_map)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    gradient_checkpointing=gradient_checkpointing,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    do_eval=True,
    evaluation_strategy="steps"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if not tokenizer.chat_template:
    tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

In [28]:
def special_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"{instruction_prompt_template}{example['instruction'][i]}\n{response_template} {example['output'][i]}"
        output_texts.append(text)
    return output_texts


def normal_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        chat_temp = [
            {"role": "system", "content": example["instruction"][i]},
            {"role": "assistant", "content": example["output"][i]},
        ]
        text = tokenizer.apply_chat_template(chat_temp, tokenize=False)
        output_texts.append(text)
    return output_texts

In [29]:
if use_special_template:
    formatting_func = special_formatting_prompts
    if use_llama_like_model:
        response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer)
    else:
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)
else:
    formatting_func = normal_formatting_prompts

In [30]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["prescription"]["train_dataset"],
    eval_dataset=dataset_dict["prescription"]["eval_dataset"],
    peft_config=peft_config,
    formatting_func=formatting_func,
    data_collator=collator,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [31]:
if torch.cuda.device_count() > 1:
  model.is_parallelizable = True
  model.model_parallel = True

# Train model
trainer.train()

# Save fine tuned Lora Adaptor
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,0.306700,0.229847
20,0.215800,0.205566
30,0.228600,0.194928
40,0.193300,0.202306
50,0.169700,0.207046
60,0.183700,0.202105
70,0.180000,0.207851
80,0.162000,0.189653
90,0.161700,0.206336
100,0.158600,0.189224


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:155: UserWarning: Could not find response key `[26307, 28747]` in the following instance: <s> "### Human:"Here is the clinical text from the doctor in the delimiter. clinical text: <<<Clinic date 10/12/2019   r.e. Llewleyn Richards. D.O.B. 23.10.1980   Diagnosis:    Symptomatic structural epilepsy secondary to         Significant traumatic brain injury 2005   Seizure type and frequency:       Focal to bilateral convulsive seizures  , last event October 2019   Current anti-epi

In [ ]:
# import torch
# import gc


# def clear_hardwares():
#     torch.clear_autocast_cache()
#     torch.cuda.ipc_collect()
#     torch.cuda.empty_cache()
#     gc.collect()


# clear_hardwares()
# clear_hardwares()

In [32]:
def generate(model, prompt: str, kwargs):
    tokenized_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

    prompt_length = len(tokenized_prompt.get("input_ids")[0])

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**tokenized_prompt, **kwargs) if kwargs else model.generate(**tokenized_prompt)
        output = tokenizer.decode(output_tokens[0][prompt_length:], skip_special_tokens=True)

    return output

In [33]:
new_model

'mistralee-prescription-json-generator'

In [34]:
new_model_path = "/content/mistralee-prescription-json-generator"

In [35]:
ft_model = AutoModelForCausalLM.from_pretrained(new_model_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading adapter weights from /content/mistralee-prescription-json-generator led to unexpected keys not found in the model:  ['model.layers.0.self_attn.o_proj.lora_A.default.weight', 'model.layers.0.self_attn.o_proj.lora_B.default.weight', 'model.layers.0.mlp.gate_proj.lora_A.default.weight', 'model.layers.0.mlp.gate_proj.lora_B.default.weight', 'model.layers.0.mlp.up_proj.lora_A.default.weight', 'model.layers.0.mlp.up_proj.lora_B.default.weight', 'model.layers.0.mlp.down_proj.lora_A.default.weight', 'model.layers.0.mlp.down_proj.lora_B.default.weight', 'model.layers.1.self_attn.o_proj.lora_A.default.weight', 'model.layers.1.self_attn.o_proj.lora_B.default.weight', 'model.layers.1.mlp.gate_proj.lora_A.default.weight', 'model.layers.1.mlp.gate_proj.lora_B.default.weight', 'model.layers.1.mlp.up_proj.lora_A.default.weight', 'model.layers.1.mlp.up_proj.lora_B.default.weight', 'model.layers.1.mlp.down_proj.lora_A.default.weight', 'model.layers.1.mlp.down_proj.lora_B.default.weight', 'model.

In [36]:
peft_model = PeftModel.from_pretrained(ft_model, new_model)
# del base_model

In [38]:
sample = dataset_dict["prescription"]["eval_dataset"][9]
if use_special_template:
    prompt = f"{instruction_prompt_template}{sample['instruction']}\n{response_template}"
else:
    chat_temp = [{"role": "system", "content": sample["instruction"]}]
    prompt = tokenizer.apply_chat_template(chat_temp, tokenize=False, add_generation_prompt=True)

prompt

'"### Human:"Here is the clinical text from the doctor in the delimiter. clinical text: <<<Diagnosis: Refractory epilepsy  Medication: Levetiracetam 1250mg bd    Lamotrigine 100mg bd    I reviewed this woman today in clinic. Since increasing the levetiracetam there has been no change in her   seizure   frequency. She has around 3-5 seizure  s   a week.  In June her lamotrigine level was measured and was slightly high. She had a rash at the time but I am not sure if the lamotrigine caused this as she has been taking lamotrigine for several years. Her mood is not brilliant and she is finding that she is sleeping a lot in the day whilst being awake during the night.  We talked about some treatment options. I would advise switching the levetiracetam to brivetiracetam (100mg bd) this would be a straight switch. We also discussed some sleep hygiene measures and I have included a leaflet on some advice on sleeping. Also I would be grateful if you could prescribe melatonin 8mg on for 6 weeks t

In [43]:
gen_kwargs = {"max_new_tokens": 128}
generated_texts = generate(model=peft_model, prompt=prompt, kwargs=gen_kwargs)
print(generated_texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [44]:
generated_texts

"{'entity': 'Prescription', 'start_index': '123', 'end_index': '140', 'text': 'Lamotrigine 100mg bd', 'attributes': {'DrugName': 'lamotrigine', 'DrugDose': '100', 'DoseUnit': 'mg', 'Frequency': '2'}} {'entity': 'Prescription', 'start_index': '141', 'end_index': '168', 'text': 'Levetiracetam 1250"

In [45]:
sample["output"]

{'entity': 'Prescription',
 'start_index': '32',
 'end_index': '73',
 'text': 'Medication:       Levetiracetam 1250mg bd',
 'attributes': {'DrugName': 'levetiracetam',
  'DrugDose': '1250',
  'DoseUnit': 'mg',
  'Frequency': '2'}}

In [ ]:
from ast import literal_eval

In [ ]:
gen_text = literal_eval(generated_texts)

In [ ]:
gen_text

{'entity': 'Prescription',
 'start_index': '104',
 'end_index': '123',
 'text': 'lamotrigine-150mg-bd',
 'attributes': {'DrugName': 'lamotrigine',
  'DrugDose': '150',
  'DoseUnit': 'mg',
  'Frequency': '2'}}

In [ ]:
seizure_model_name = ""